# Dragon Test:

In [4]:
import torch
from transformers import AutoTokenizer
from dragon_baseline.architectures.clf_multi_head import (
    AutoModelForMultiHeadSequenceClassification,
    AutoModelForMultiHeadSequenceClassificationConfig)


model_name = "bert-base-multilingual-cased"
num_classes_per_label = [2, 3, 4]  

config = AutoModelForMultiHeadSequenceClassificationConfig(
    pretrained_model_name_or_path=model_name,
    num_classes_per_label=num_classes_per_label,)
model = AutoModelForMultiHeadSequenceClassification(config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
batch = tokenizer(["This is a test sentence and just randomly used to check the performance."], return_tensors="pt", padding=True, truncation=True)

labels = torch.tensor([[1, 2, 2]])
model.eval()
with torch.no_grad():
    outputs = model(**batch, labels=labels)

print("Loss:", outputs.loss)
print("Logits per head:")
for i, head_logits in enumerate(outputs.logits):
    # print(f"Head {i+1} logits:", head_logits)
    probs = torch.softmax(head_logits, dim=1)
    print(f"Head {i+1} logits:", probs)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: tensor(0.3859)
Logits per head:
Head 1 logits: tensor([[0.5549, 0.4451]])
Head 2 logits: tensor([[0.2881, 0.3753, 0.3365]])
Head 3 logits: tensor([[0.3876, 0.2133, 0.2071, 0.1920]])


In [5]:
import torch
from transformers import AutoTokenizer
from dragon_baseline.architectures.ner_multi_head import AutoModelForMultiHeadTokenClassification, AutoModelForMultiHeadTokenClassificationConfig
import numpy as np

config = AutoModelForMultiHeadTokenClassificationConfig(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    num_labels=3  # For example: Person, Organization, Location
)
model = AutoModelForMultiHeadTokenClassification(config)
model.eval()


tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
text = "Barack Obama visited Microsoft headquarters in Seattle."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).long()

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
id2label = model.id2label

print("\nToken predictions (multi-label):")
for i, token in enumerate(tokens):
    pred_vec = preds[0][i].tolist()
    if any(pred_vec):
        label_info = [f"Label {j}" for j, val in enumerate(pred_vec) if val == 1]
        print(f"{token:15} → {label_info}")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Token predictions (multi-label):
[CLS]           → ['Label 2']
Obama           → ['Label 0', 'Label 2']
visited         → ['Label 2']
headquarters    → ['Label 2']
in              → ['Label 1', 'Label 2']
Seattle         → ['Label 1']
.               → ['Label 2']
[SEP]           → ['Label 2']


In [6]:
import torch
from transformers import AutoTokenizer
from dragon_baseline.architectures.reg_multi_head import AutoModelForMultiHeadSequenceRegression, AutoModelForMultiHeadSequenceRegressionConfig

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
texts = ["This is a test sentence.", "Here is another one."]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

num_labels = 3
labels = torch.tensor([[0.5, float('nan'), 1.0], [1.2, 0.8, float('nan')]], dtype=torch.float)
config = AutoModelForMultiHeadSequenceRegressionConfig(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    num_labels=num_labels,
    filter_targets=True)
model = AutoModelForMultiHeadSequenceRegression(config)
model.eval()

with torch.no_grad():
    output = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        labels=labels,
        return_dict=True)

print("Logits:\n", output.logits)
print("\nLoss:", output.loss)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits:
 tensor([[ 0.0093, -0.3596,  0.0265],
        [ 0.0021, -0.2745, -0.0995]])

Loss: tensor(0.9445)
